<a href="https://colab.research.google.com/github/LeonardoRoig/TECH_5/blob/main/ETL_Modelo_Streamlit_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 ETL_Modelo_Streamlit_A
Notebook que treina o modelo, gera ranking e prints do recrutador, e no final exporta os arquivos `.py` e `requirements.txt` para deploy no Streamlit Cloud.

## 1) Imports e Configuração

In [1]:

import os, re, json
import pandas as pd, numpy as np
from typing import List, Optional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option("display.max_colwidth", 300)

# Colunas padrão
ID_VAGA_COL, ID_CAND_COL, TARGET_COL = "id_vaga", "id_candidato", "target"

# Caminhos padrão
TRAIN_PATH, PENDING_PATH, EXPORT_DIR = "/content/aprovados_reprovados.csv", "/content/nao_classificados.csv", "/content/artifacts"
os.makedirs(EXPORT_DIR, exist_ok=True)


## 2) Leitura dos Dados

In [2]:

df_train = pd.read_csv(TRAIN_PATH)
df_pending = pd.read_csv(PENDING_PATH)
print("Treino:", df_train.shape, "Pendentes:", df_pending.shape)
df_train.head()


Treino: (10110, 31) Pendentes: (34961, 31)


,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,...,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos
0,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em projetos de Rollout e implementação SAP ECC\n• Inglês mandatório\n• Remoto (Em alguns momentos / fases do projeto deverá estar presente na planta do cliente em Campinas/SP),Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênior com experiencia em projetos de Rollout e implementação SAP ECC\n• Inglês mandatório\n• Remoto (Em alguns momentos / fases do projeto deverá estar presente na planta do cliente em Campinas/SP),...,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
1,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em projetos de Rollout e implementação SAP ECC\n• Inglês mandatório\n• Remoto (Em alguns momentos / fases do projeto deverá estar presente na planta do cliente em Campinas/SP),Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênior com experiencia em projetos de Rollout e implementação SAP ECC\n• Inglês mandatório\n• Remoto (Em alguns momentos / fases do projeto deverá estar presente na planta do cliente em Campinas/SP),...,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
2,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em projetos de Rollout e implementação SAP ECC\n• Inglês mandatório\n• Remoto (Em alguns momentos / fases do projeto deverá estar presente na planta do cliente em Campinas/SP),Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênior com experiencia em projetos de Rollout e implementação SAP ECC\n• Inglês mandatório\n• Remoto (Em alguns momentos / fases do projeto deverá estar presente na planta do cliente em Campinas/SP),...,Não informado,Não informado,Não informado,Mestrado Completo,Fluente,Fluente,Não informado,Não informado,Não informado,Engenharia da Computação
3,5184,Consultor PP/QM Sênior,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Sênior,Fluente,Nenhum,• Consultor PP/QM Sênior com experiencia em projetos de Rollout e implementação SAP ECC\n• Inglês mandatório\n• Remoto (Em alguns momentos / fases do projeto deverá estar presente na planta do cliente em Campinas/SP),Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênior com experiencia em projetos de Rollout e implementação SAP ECC\n• Inglês mandatório\n• Remoto (Em alguns momentos / fases do projeto deverá estar presente na planta do cliente em Campinas/SP),...,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado
4,5183,ANALISTA PL/JR C/ SQL,"Morris, Moran and Dodson",Não,Ensino Superior Completo,Analista,Nenhum,Intermediário,"Requisitos mandatórios:\n\no Conhecimentos Técnicos: Conhecimento SQL, e algum conhecimento de SAP SD\no Competências Interpessoais: Relacionamento interpessoal, foco no cliente, trabalho em equipe, excelente\ncomunicação, e adaptabilidade.\n\nRequisitos desejáveis:\no Idiomas: Espanhol nível in...","Descrição – Atividades:\n\no Monitoramento das interfaces KDP\no Monitoramento sistema B2B durante o dia\no Monitoramento do Whatsapp durante o dia\no Monitoramento da subida de pedidos (KDP, WAE e B2B)\no Suporte para equipe comercial quando tem problemas em algum cliente no WAE\no Suporte Apli...",...,Não informado,Não informado,Não informado,Pós Graduação Cursando,Básico,Básico,Não

## 3) Pré-processamento

In [3]:

X, y = df_train.drop(columns=[TARGET_COL]), df_train[TARGET_COL]
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
for c in [ID_VAGA_COL, ID_CAND_COL]:
    if c in cat_cols: cat_cols.remove(c)
preprocessor = ColumnTransformer([("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)], remainder="passthrough")


## 4) Treino e Avaliação

In [4]:

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
model = Pipeline([("preprocess", preprocessor), ("clf", RandomForestClassifier(n_estimators=300, random_state=42))])
model.fit(X_train, y_train)
y_pred, y_proba = model.predict(X_val), model.predict_proba(X_val)[:,1]
print(classification_report(y_val, y_pred))
print("ROC AUC:", roc_auc_score(y_val, y_proba))


              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      1534
         1.0       1.00      0.98      0.99       488

    accuracy                           0.99      2022
   macro avg       1.00      0.99      0.99      2022
weighted avg       0.99      0.99      0.99      2022

ROC AUC: 0.9998690875670592


A acurácia do modelo navalidação é de 99%.

## 5) Predição e Ranking

In [16]:
df_pred = df_pending.copy()
df_pred["score"] = model.predict_proba(df_pred)[:,1]
df_pred["rank"] = df_pred.groupby(ID_VAGA_COL)["score"].rank(ascending=False, method="first")
ranking = df_pred[df_pred["rank"] <= 10].sort_values([ID_VAGA_COL,"rank"]).reset_index(drop=True)
pd.set_option('display.max_columns', None)
display(ranking.head(200))
#display(ranking)

,id_vaga,inf_titulo_vaga,inf_cliente,inf_vaga_sap,perfil_nivel_academico,perfil_nivel profissional,perfil_nivel_ingles,perfil_nivel_espanhol,perfil_competencia_tecnicas_e_comportamentais,perfil_principais_atividades,titulo,id_candidato,nome,data_candidatura,recrutador,situacao_candidado,target,objetivo_profissional,titulo_profissional,area_atuacao,conhecimentos_tecnicos,qualificacoes,certificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,cargo_atual,nivel_profissional,outro_idioma,cursos,score,rank
0,3,Arquiteto de Sistemas SR,Barnes-Woods,Não,Ensino Técnico Completo,Especialista,Fluente,Nenhum,"More than 10 years retail experience Oracle technologies and retail areas as ORWMS, ORSIM,ORMS, OREIM.\nEnglish and Portuguese mandatory",Oracle Retail Senior Solution Architect / Integration Arqchitect E2E,Arquiteto de Sistemas SR,12598,Sr. Luiz Fernando Fernandes,06-12-2018,Liz Freitas,Encaminhado ao Requisitante,NaN,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,0.253333,1.0
1,4,Analista de Projetos SR,Barnes-Woods,Não,Ensino Técnico Completo,Analista,Avançado,Nenhum,More than 5 years retail experience Oracle technologies ORSIM\nPortuguese should be mandatory if Brazil location,Oracle Retail SIM specialist - SME,Analista de Projetos SR,12618,Dra. Kamilly Nascimento,07-12-2018,Aylla Leão,Encaminhado ao Requisitante,NaN,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,0.256667,1.0
2,5,Analista de Sistemas SR,Callahan-Hall,Não,Ensino Técnico Completo,Analista,Nenhum,Nenhum,"Substituição Wesley\nProfissional com mais de 10 anos de experiência de desenvolvendo aplicações web de escala com tecnologia Java,.Net ou similares; mais de 3 anos de experiência definindo arquitetura de aplicações web de escala com APIs REST; 2+ anos de experiência administrando aplicações cloud em AWS;\nExperiência trabalhando com metodologias Agile (por ex: SCRUM e Kanban)",Analista desenvolvedor,Analista de Sistemas SR,12626,Luiz Felipe Moraes,10-12-2018,Liz Freitas,Encaminhado ao Requisitante,NaN,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,0.236667,1.0
3,5,Analista de Sistemas SR,Callahan-Hall,Não,Ensino Técnico Completo,Analista,Nenhum,Nenhum,"Substituição Wesley\nProfissional com mais de 10 anos de experiência de desenvolvendo aplicações web de escala com tecnologia Java,.Net ou similares; mais de 3 anos de experiência definindo arquitetura de aplicações web de escala com APIs REST; 2+ anos de experiência administrando aplicações cloud em AWS;\nExperiência trabalhando com metodologias Agile (por ex: SCRUM e Kanban)",Analista desenvolvedor,Analista de Sistemas SR,12624,Dra. Aylla Monteiro,10-12-2018,Liz Freitas,Encaminhado ao Requisitante,NaN,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,0.236667,2.0
4,5,Analista de Sistemas SR,Callahan-Hall,Não,Ensino Técnico Completo,Analista,Nenhum,Nenhum,"Substituição Wesley\nProfissional com mais de 10 anos de experiência de desenvolvendo aplicações web de escala com tecnologia Java,.Net ou similares; mais de 3 anos de experiência definindo arquitetura de aplicações web de escala com APIs REST; 2+ anos de experiência administrando aplicações cloud em AWS;\nExperiência trabalhando com metodologias Agile (por ex: SCRUM e Kanban)",Analista desenvolvedor,Analista de Sistemas SR,11299,Cauê Teixeira,06-12-2018,Liz Freitas,Encaminhado ao Requisitante,NaN,Scrum,Scrum,Não informado,Não informado,Não informado,Não informado,Não 

## 6) Funções auxiliares para Similaridade e Print do Recrutador

In [6]:

def top_terms_overlap(job_text, cand_text, k=8):
    tok = lambda s: set(re.findall(r"[\w\-\+]+", str(s).lower()))
    jset, cset = tok(job_text), tok(cand_text)
    return {"match_terms": list(jset & cset)[:k], "missing_terms": list(jset - cset)[:k]}

def local_similarity(job_text, cand_text):
    vec = TfidfVectorizer(max_features=2000)
    tfidf = vec.fit_transform([str(job_text), str(cand_text)])
    return float(cosine_similarity(tfidf[0], tfidf[1])[0][0])


## 7) Exportar `train_and_rank.py`

In [7]:

train_code = """
import re, json
import pandas as pd, numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

ID_VAGA_COL, ID_CAND_COL, TARGET_COL = "id_vaga", "id_candidato", "target"

def top_terms_overlap(job_text, cand_text, k=8):
    tok = lambda s: set(re.findall(r"[\w\-\+]+", str(s).lower()))
    jset, cset = tok(job_text), tok(cand_text)
    return {"match_terms": list(jset & cset)[:k], "missing_terms": list(jset - cset)[:k]}

def local_similarity(job_text, cand_text):
    vec = TfidfVectorizer(max_features=2000)
    tfidf = vec.fit_transform([str(job_text), str(cand_text)])
    return float(cosine_similarity(tfidf[0], tfidf[1])[0][0])

def run_pipeline(train_path, pending_path, export_dir="/content/artifacts", top_k=10):
    df_train, df_pending = pd.read_csv(train_path), pd.read_csv(pending_path)
    X, y = df_train.drop(columns=[TARGET_COL]), df_train[TARGET_COL]
    cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
    for c in [ID_VAGA_COL, ID_CAND_COL]:
        if c in cat_cols: cat_cols.remove(c)
    preprocessor = ColumnTransformer([("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)], remainder="passthrough")
    pipe = Pipeline([("preprocess", preprocessor), ("clf", RandomForestClassifier(n_estimators=300, random_state=42))])
    pipe.fit(X, y)
    df_pred = df_pending.copy()
    df_pred["score"] = pipe.predict_proba(df_pred)[:,1]
    df_pred["rank"] = df_pred.groupby(ID_VAGA_COL)["score"].rank(ascending=False, method="first")
    ranking = df_pred[df_pred["rank"] <= top_k].sort_values([ID_VAGA_COL,"rank"]).reset_index(drop=True)
    ranking.to_csv(f"{export_dir}/ranking_por_vaga.csv", index=False)
    cards = ranking[[ID_VAGA_COL, ID_CAND_COL, "score", "rank"]].to_dict(orient="records")
    with open(f"{export_dir}/cards_recrutador.json", "w", encoding="utf-8") as f:
        json.dump(cards, f, ensure_ascii=False, indent=2)
    return ranking
"""

with open(os.path.join(EXPORT_DIR, "train_and_rank.py"), "w", encoding="utf-8") as f:
    f.write(train_code)
print("Arquivo train_and_rank.py exportado!")


Arquivo train_and_rank.py exportado!


<>:14: SyntaxWarning: invalid escape sequence '\w'
<>:14: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipython-input-2677638376.py:14: SyntaxWarning: invalid escape sequence '\w'
  tok = lambda s: set(re.findall(r"[\w\-\+]+", str(s).lower()))


## 8) Exportar `app.py`

In [8]:

app_code = """
import json, pandas as pd, streamlit as st
from train_and_rank import run_pipeline

st.set_page_config(page_title="Netflix das Vagas", layout="wide")
st.title("🎬 Netflix das Vagas — Top 10 Candidatos por Vaga")

train_file = st.file_uploader("CSV de treino (aprovados/reprovados)", type=["csv"])
pending_file = st.file_uploader("CSV de pendentes (não classificados)", type=["csv"])

if train_file and pending_file:
    with open("train.csv","wb") as f: f.write(train_file.getbuffer())
    with open("pending.csv","wb") as f: f.write(pending_file.getbuffer())
    ranking = run_pipeline("train.csv", "pending.csv", export_dir=".")
    st.success("Pipeline executado!")

    vagas = sorted(ranking["id_vaga"].unique().tolist())
    vaga_sel = st.sidebar.selectbox("Selecione a vaga", vagas)
    top = ranking[ranking["id_vaga"]==vaga_sel].sort_values("rank")
    st.subheader(f"Top {len(top)} candidatos para a vaga {vaga_sel}")

    cols = st.columns(5)
    for i, (_, row) in enumerate(top.iterrows()):
        col = cols[i % 5]
        with col:
            st.markdown(f"### 👤 Candidato {row['id_candidato']}")
            st.metric("Score (modelo)", f"{row['score']:.3f}")
            st.caption(f"Rank: {row['rank']}")
    with st.expander("Tabela completa da vaga"):
        st.dataframe(top)
else:
    st.info("Faça upload dos dois arquivos CSV para rodar o pipeline.")
"""

with open(os.path.join(EXPORT_DIR, "app.py"), "w", encoding="utf-8") as f:
    f.write(app_code)
print("Arquivo app.py exportado!")


Arquivo app.py exportado!


## 9) Exportar `requirements.txt`

In [9]:

reqs = """pandas==2.2.2
numpy==1.26.4
scikit-learn==1.4.2
streamlit==1.32.0
"""

with open(os.path.join(EXPORT_DIR, "requirements.txt"), "w", encoding="utf-8") as f:
    f.write(reqs)
print("Arquivo requirements.txt exportado!")


Arquivo requirements.txt exportado!
